In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle
from datetime import datetime
import os

In [ ]:
# 1. FUNCION DE IMPUTACION ESTACIONAL

def imputar_datos_estacionales(df):
    """
    Imputa valores NaN en la columna 'value' basándose en la media por hora y día de la semana.
    Si no hay NaNs, retorna el mismo DataFrame.
    """
    if df['value'].isna().sum() == 0:
        print(" No se encontraron valores NaN en 'value'.")
        return df

    print(" Se encontraron valores NaN. Imputando...")

    df['weekday'] = df['datetime'].dt.dayofweek  # Lunes=0, Domingo=6
    df['hour'] = df['datetime'].dt.hour

    media_estacional = (
        df.groupby(['weekday', 'hour'])['value']
        .mean()
        .reset_index()
        .rename(columns={'value': 'media_estacional'})
    )

    df = df.merge(media_estacional, on=['weekday', 'hour'], how='left')
    df['value'] = df['value'].fillna(df['media_estacional'])
    df = df.drop(columns=['media_estacional'])

    print(" Imputación completada usando estacionalidad.")
    return df

In [ ]:
# 2. FUNCION DE ESCALADO Y GUARDADO DEL SCALER

def escalar_datos(df, save_path='scaler.pkl'):
    """
    Escala la columna 'value' usando StandardScaler y guarda el objeto scaler como pickle.
    """
    scaler = StandardScaler()
    df['value_scaled'] = scaler.fit_transform(df[['value']])

    with open(save_path, 'wb') as f:
        pickle.dump(scaler, f)

    print(f" Datos escalados y scaler guardado en '{save_path}'.")
    return df


In [ ]:
# 3. FUNCION PARA DIVIDIR EN TRAIN Y TEST

def dividir_train_test(df, test_size=0.2):
    """
    Divide los datos en entrenamiento y prueba respetando el orden temporal.
    """
    df = df.sort_values('datetime')
    n_total = len(df)
    n_test = int(n_total * test_size)
    n_train = n_total - n_test

    df_train = df.iloc[:n_train].reset_index(drop=True)
    df_test = df.iloc[n_train:].reset_index(drop=True)

    print(f" División completada: {len(df_train)} train / {len(df_test)} test")
    return df_train, df_test



In [ ]:
# Cargar los datos
df = pd.read_csv("demanda.csv", parse_dates=['datetime'])
print("Datos cargados:")
print(df.head())

# Imputación
df = imputar_datos_estacionales(df)
print("Después de imputación:")
print(df.head())

# Escalado
df = escalar_datos(df)
print("Después de escalado:")
print(df[['datetime', 'value', 'value_scaled']].head())

# División train/test
df_train, df_test = dividir_train_test(df)
print(f" Train shape: {df_train.shape}")
print(f" Test shape: {df_test.shape}")
